In [5]:
#import dependencies
import pandas as pd
from scipy.interpolate import interp1d
import numpy as np
from datetime import timedelta
import numpy as np
from sklearn.metrics import mean_squared_error
import math

#Library to plot data
import plotly.graph_objects as go

#import csv data for plotting
MAPE_data = pd.read_csv('https://raw.githubusercontent.com/Dan-Irl/MVEX-Matematiska-modeller-f-r-smittspridning/main/MAPE_all_Models.csv')
ISalpha = pd.read_csv('https://raw.githubusercontent.com/Dan-Irl/MVEX-Matematiska-modeller-f-r-smittspridning/main/ISalpha_all_Models.csv')
Totaladmissions = pd.read_csv('https://raw.githubusercontent.com/Dan-Irl/MVEX-Matematiska-modeller-f-r-smittspridning/main/Totaladmissions_all_models.csv')
logistiskadata = pd.read_csv('https://raw.githubusercontent.com/Dan-Irl/MVEX-Matematiska-modeller-f-r-smittspridning/main/plottingDataLogistiska/logistiska_plot.csv')
logistiskadata_mobilitet = pd.read_csv('https://raw.githubusercontent.com/Dan-Irl/MVEX-Matematiska-modeller-f-r-smittspridning/main/plottingDataLogistiska/logistiska_mobilitet_plot.csv')
logistiskadata_tak = pd.read_csv('https://raw.githubusercontent.com/Dan-Irl/MVEX-Matematiska-modeller-f-r-smittspridning/main/plottingDataLogistiska/logistiska_tak_plot.csv')
logistiskadata_mobilitet_tak = pd.read_csv('https://raw.githubusercontent.com/Dan-Irl/MVEX-Matematiska-modeller-f-r-smittspridning/main/plottingDataLogistiska/logistiska_mobilitet_tak_plot.csv')
admission_data = pd.read_json('https://raw.githubusercontent.com/Dan-Irl/MVEX-Matematiska-modeller-f-r-smittspridning/main/Inskrivningsdata.json')

#Number of data point and time array for plotting
data_points_MAPE = 109
t_data_points_MAPE = np.linspace(1,data_points_MAPE,data_points_MAPE)

#Interpolating weekly MAPE SIR data to daily
temp_MAPE_SIR_weekly = MAPE_data['MAPE_SIR_weekly']
temp_MAPE_SIR_weekly.truncate(0,14,copy = False)

SIR_MAPE_week2daily = []
for week in range(len(temp_MAPE_SIR_weekly)-1):
    SIR_MAPE_week2daily.extend(np.linspace(temp_MAPE_SIR_weekly[week],temp_MAPE_SIR_weekly[week+1],8)[:7])
     
#Interpolating weekly ISalpha SIR data to daily
temp_ISalpha_SIR_weekly = ISalpha['ISalpha_SIR_weekly']
temp_ISalpha_SIR_weekly.truncate(0,14,copy = False)

SIR_ISalpha_week2daily = []
for week in range(len(temp_ISalpha_SIR_weekly)-1):
    SIR_ISalpha_week2daily.extend(np.linspace(temp_ISalpha_SIR_weekly[week],temp_ISalpha_SIR_weekly[week+1],8)[:7])
    
    
#Interpolating weekly ISalpha SIR data to daily
temp_totaladmissions_SIR_weekly = Totaladmissions['totalAdmissions_SIR_weekly']
temp_totaladmissions_SIR_weekly.truncate(0,14,copy = False)

SIR_totaladmissions_week2daily = []
for week in range(len(temp_totaladmissions_SIR_weekly)-1):
    SIR_totaladmissions_week2daily.extend(np.linspace(temp_totaladmissions_SIR_weekly[week],temp_totaladmissions_SIR_weekly[week+1],8)[:7])    
    
datapoint_logistiska = 153

In [6]:
#plotting off MAPE data

MAPEfig1 = go.Figure()

#SIR MAPE week to daily
MAPEfig1.add_trace(go.Scatter(x=pd.Series(pd.date_range(pd.to_datetime('2020-03-01')+timedelta(days=42), freq="D", periods=data_points_MAPE)), 
                    y=SIR_MAPE_week2daily,
                    mode='lines',
                    name='SIR veckolig modell (interpolerad)'))

#SIR MAPE daily
MAPEfig1.add_trace(go.Scatter(x=pd.Series(pd.date_range(pd.to_datetime('2020-03-01')+timedelta(days=42), freq="D", periods=data_points_MAPE)), 
                    y=MAPE_data['MAPE_SIR_daily'],
                    mode='lines',
                    name='SIR daglig modell'))



MAPEfig1.update_layout(
                    title='MAPE mot ökande inskrivningsdata',
                    xaxis_title='Date',
                    yaxis_title='MAPE %',
                    autosize=False,
                    width=1000,
                    xaxis_range=[pd.to_datetime('2020-03-01')+timedelta(days=42),pd.to_datetime('2020-03-01')+timedelta(days=42)+timedelta(days=data_points_MAPE)],
                    yaxis_range=[0,100],
                    font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="RebeccaPurple"
    )
) 
MAPEfig1.show()

MAPEfig2 = go.Figure()


#logistiska MAPE
MAPEfig2.add_trace(go.Scatter(x=pd.Series(pd.date_range(pd.to_datetime('2020-03-01')+timedelta(days=42), freq="D", periods=data_points_MAPE)), 
                    y=MAPE_data['MAPE_logistiska'],
                    mode='lines',
                    name='Logistisk modell'))

#logistiska mobilitet MAPE
MAPEfig2.add_trace(go.Scatter(x=pd.Series(pd.date_range(pd.to_datetime('2020-03-01')+timedelta(days=42), freq="D", periods=data_points_MAPE)), 
                    y=MAPE_data['MAPE_logistiska_mobilitet'],
                    mode='lines',
                    name='Logistisk modell med mobilitet'))

#logistiska tak MAPE
MAPEfig2.add_trace(go.Scatter(x=pd.Series(pd.date_range(pd.to_datetime('2020-03-01')+timedelta(days=42), freq="D", periods=data_points_MAPE)), 
                    y=MAPE_data['MAPE_logistiska_tak'],
                    mode='lines',
                    name='Logistisk modell med tak'))

#logistiska mobilitet tak MAPE
MAPEfig2.add_trace(go.Scatter(x=pd.Series(pd.date_range(pd.to_datetime('2020-03-01')+timedelta(days=42), freq="D", periods=data_points_MAPE)),
                    y=MAPE_data['MAPE_logistiska_mobilitet_tak'],
                    mode='lines',
                    name='Logistisk modell med mobilitet och tak'))

MAPEfig2.update_layout(
                    title='MAPE mot ökande inskrivningsdata',
                    xaxis_title='Date',
                    yaxis_title='MAPE %',
                    autosize=False,
                    width=1000,
                    xaxis_range=[pd.to_datetime('2020-03-01')+timedelta(days=42),pd.to_datetime('2020-03-01')+timedelta(days=42)+timedelta(days=data_points_MAPE)],
                    
                    font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="RebeccaPurple"
    )
) 
MAPEfig2.show()

In [19]:
#plotting of ISalpha data

ISalphafig1 = go.Figure()

#SIR ISalpha week to daily
ISalphafig1.add_trace(go.Scatter(x=pd.Series(pd.date_range(pd.to_datetime('2020-03-01')+timedelta(days=42), freq="D", periods=data_points_MAPE)), 
                    y=SIR_ISalpha_week2daily,
                    mode='lines',
                    name='SIR veckolig modell (interpolerad)'))

#SIR ISalpha daily
ISalphafig1.add_trace(go.Scatter(x=pd.Series(pd.date_range(pd.to_datetime('2020-03-01')+timedelta(days=42), freq="D", periods=data_points_MAPE)), 
                    y=ISalpha['ISalpha_SIR_daily'],
                    mode='lines',
                    name='SIR daglig modell'))



ISalphafig1.update_layout(
                    title='ISalpha mot ökande inskrivningsdata',
                    xaxis_title='Date',
                    yaxis_title='ISalpha score',
                    autosize=False,
                    width=1000,
                    xaxis_range=[pd.to_datetime('2020-03-01')+timedelta(days=42),pd.to_datetime('2020-03-01')+timedelta(days=42)+timedelta(days=data_points_MAPE)],
                    
                    font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="RebeccaPurple"
    )
) 
ISalphafig1.show()

ISalphafig2 = go.Figure()


#logistiska ISalpha
ISalphafig2.add_trace(go.Scatter(x=pd.Series(pd.date_range(pd.to_datetime('2020-03-01')+timedelta(days=42), freq="D", periods=data_points_MAPE)), 
                    y=ISalpha['ISalpha_logistiska'],
                    mode='lines',
                    name='Logistisk modell'))

#logistiska mobilitet ISalpha
ISalphafig2.add_trace(go.Scatter(x=pd.Series(pd.date_range(pd.to_datetime('2020-03-01')+timedelta(days=42), freq="D", periods=data_points_MAPE)), 
                    y=ISalpha['ISalpha_logistiska_mobilitet'],
                    mode='lines',
                    name='Logistisk modell med mobilitet'))

#logistiska tak ISalpha
ISalphafig2.add_trace(go.Scatter(x=pd.Series(pd.date_range(pd.to_datetime('2020-03-01')+timedelta(days=42), freq="D", periods=data_points_MAPE)), 
                    y=ISalpha['ISalpha_logistiska_tak'],
                    mode='lines',
                    name='Logistisk modell med tak'))

#logistiska mobilitet tak ISalpha
ISalphafig2.add_trace(go.Scatter(x=pd.Series(pd.date_range(pd.to_datetime('2020-03-01')+timedelta(days=42), freq="D", periods=data_points_MAPE)),
                    y=ISalpha['ISalpha_logistiska_mobilitet_tak'],
                    mode='lines',
                    name='Logistisk modell med mobilitet och tak'))

ISalphafig2.update_layout(
                    title='ISalpha mot ökande inskrivningsdata$',
                    xaxis_title='Date',
                    yaxis_title='ISalpha score',
                    autosize=False,
                    width=1000,
                    xaxis_range=[pd.to_datetime('2020-03-01')+timedelta(days=42),pd.to_datetime('2020-03-01')+timedelta(days=42)+timedelta(days=data_points_MAPE)],
                    
                    font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="RebeccaPurple"
    )
) 
ISalphafig2.show()

In [8]:
t_totaladmissions = np.linspace(21,131,111)
total_admissions = np.linspace(1516,1516,111)


Totaladmissionsfig1 = go.Figure()

#SIR total admissions week to daily
Totaladmissionsfig1.add_trace(go.Scatter(x=t_totaladmissions, 
                    y=SIR_totaladmissions_week2daily,
                    mode='lines',
                    name='SIR veckolig modell (interpolerad)'))

#SIR total admissions daily
Totaladmissionsfig1.add_trace(go.Scatter(x=t_totaladmissions, 
                    y=Totaladmissions['totalAdmissions_SIR_daily'],
                    mode='lines',
                    name='SIR daglig modell'))

#Total actual admssions
Totaladmissionsfig1.add_trace(go.Scatter(x=t_totaladmissions, 
                    y=total_admissions,
                    name='Totala inskrivningar',
                    line=dict(color='firebrick', width=3,
                              dash='dash')))



Totaladmissionsfig1.update_layout(
                    title='Totala inskrivnignar vid slutdatum mot ökande inskrivningsdata',
                    xaxis_title='Antal datapunkter',
                    yaxis_title='Totala inskrivingar',
                    autosize=False,
                    width=1000,
                    #xaxis_range=[pd.to_datetime('2020-03-01')+timedelta(days=21),pd.to_datetime('2020-03-01')+timedelta(days=42)+timedelta(days=data_points_MAPE)],
                    
                    font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="RebeccaPurple"
    )
) 
Totaladmissionsfig1.show()

Totaladmissionsfig2 = go.Figure()


#logistiska total admissions
Totaladmissionsfig2.add_trace(go.Scatter(x=t_totaladmissions, 
                    y=Totaladmissions['totalAdmissions_logistiska'],
                    mode='lines',
                    name='Logistisk modell'))

#logistiska mobilitet total admissions
Totaladmissionsfig2.add_trace(go.Scatter(x=t_totaladmissions, 
                    y=Totaladmissions['totalAdmissions_logistiska_mobilitet'],
                    mode='lines',
                    name='Logistisk modell med mobilitet'))

#logistiska tak total admissions
Totaladmissionsfig2.add_trace(go.Scatter(x=t_totaladmissions, 
                    y=Totaladmissions['totalAdmissions_logistiska_tak'],
                    mode='lines',
                    name='Logistisk modell med tak'))

#logistiska mobilitet tak MAPE
Totaladmissionsfig2.add_trace(go.Scatter(x=t_totaladmissions,
                    y=Totaladmissions['totalAdmissions_logistiska_mobilitet_tak'],
                    mode='lines',
                    name='Logistisk modell med mobilitet och tak'))

#Total actual admssions
Totaladmissionsfig2.add_trace(go.Scatter(x=t_totaladmissions, 
                    y=total_admissions,
                    name='Totala inskrivningar',
                    line=dict(color='firebrick', width=3,
                              dash='dash')))

Totaladmissionsfig2.update_layout(
                    title='Totala inskrivnignar vid slutdatum mot ökande inskrivningsdata',
                    xaxis_title='Antal datapunkter',
                    yaxis_title='Totala inskrivingar',
                    autosize=False,
                    width=1000,
                    #xaxis_range=[pd.to_datetime('2020-03-01')+timedelta(days=21),pd.to_datetime('2020-03-01')+timedelta(days=42)+timedelta(days=data_points_MAPE)],
                    
                    font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="RebeccaPurple"
    )
) 
Totaladmissionsfig2.show()

In [9]:

fig = go.Figure() 
#plots the admission data points
fig.add_trace(go.Scatter( 
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=datapoint_logistiska)), 
                    y=logistiskadata['Inskrivningsdata'],
                    mode='markers',
                    name='Inskrivningsdata'))
#plots model admissions
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=datapoint_logistiska)), 
                    y=logistiskadata['Optimala värden'],
                    mode='lines',
                    name='predikterade inskrivningar'))
#plots lower confidence interval
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=datapoint_logistiska)), 
                    y=logistiskadata['Undre konfidens'],
                    mode='lines',
                    name='Undre konfidensintervall',
                    line = dict( width=2, dash='dash')))
#plots Upper confidence interval
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=datapoint_logistiska)), 
                    y=logistiskadata['Övre konfidens'],
                    mode='lines',
                    name='Övre konfidensintervall',
                    line = dict(width=2, dash='dash')))
#plots lower prediction interval
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=datapoint_logistiska)), 
                    y=logistiskadata['Undre prediktion'],
                    mode='lines',
                    name='Undre prediktionsintervall',
                    line = dict(width=2, dash='dot')))
#plots upper prediction interval
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=datapoint_logistiska)), 
                    y=logistiskadata['Övre prediktion'],
                    mode='lines',
                    name='Övre prediktionsintervall',
                    line = dict(width=2, dash='dot')))
#plots RMSE annotation box
fig.add_annotation(x=pd.to_datetime("2020-03-01"),y=0, 
                   text=f"RMSE: {math.floor(mean_squared_error(logistiskadata['Inskrivningsdata'],logistiskadata['Optimala värden'], squared = False))}",
                   showarrow=False,
                   bordercolor="black",
                   xshift=48,
                   yshift=270
                  )
fig.update_layout(
                    title='Logistiska modellen',
                    xaxis_title='Datum',
                    yaxis_title='Inskrivningar',
                    autosize=False,
                    width=1000,
                    xaxis_range=["2020-03-01",pd.to_datetime('2020-03-01')+timedelta(days=datapoint_logistiska)],
                    
                    font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="RebeccaPurple"
                    )
)
fig.show()

#fig.write_image("./logistiskaKumulativaResultat.svg")

In [10]:

fig = go.Figure() 
#plots the admission data points
fig.add_trace(go.Scatter( 
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=datapoint_logistiska)), 
                    y=logistiskadata_mobilitet['Inskrivningsdata'],
                    mode='markers',
                    name='Inskrivningsdata'))
#plots model admissions
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=datapoint_logistiska)), 
                    y=logistiskadata_mobilitet['Optimala värden'],
                    mode='lines',
                    name='predikterade inskrivningar'))
#plots lower confidence interval
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=datapoint_logistiska)), 
                    y=logistiskadata_mobilitet['Undre konfidens'],
                    mode='lines',
                    name='Undre konfidensintervall',
                    line = dict( width=2, dash='dash')))
#plots Upper confidence interval
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=datapoint_logistiska)), 
                    y=logistiskadata_mobilitet['Övre konfidens'],
                    mode='lines',
                    name='Övre konfidensintervall',
                    line = dict(width=2, dash='dash')))
#plots lower prediction interval
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=datapoint_logistiska)), 
                    y=logistiskadata_mobilitet['Undre prediktion'],
                    mode='lines',
                    name='Undre prediktionsintervall',
                    line = dict(width=2, dash='dot')))
#plots upper prediction interval
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=datapoint_logistiska)), 
                    y=logistiskadata_mobilitet['Övre prediktion'],
                    mode='lines',
                    name='Övre prediktionsintervall',
                    line = dict(width=2, dash='dot')))
#plots RMSE annotation box
fig.add_annotation(x=pd.to_datetime("2020-03-01"),y=0, 
                   text=f"RMSE: {math.floor(mean_squared_error(logistiskadata_mobilitet['Inskrivningsdata'],logistiskadata_mobilitet['Optimala värden'], squared = False))}",
                   showarrow=False,
                   bordercolor="black",
                   xshift=48,
                   yshift=270
                  )
fig.update_layout(
                    title='Logistiska modellen med mobilitetsdata',
                    xaxis_title='Datum',
                    yaxis_title='Inskrivningar',
                    autosize=False,
                    width=1000,
                    xaxis_range=["2020-03-01",pd.to_datetime('2020-03-01')+timedelta(days=datapoint_logistiska)],
                    
                    font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="RebeccaPurple"
                    )
)
fig.show()
#fig.write_image("./logistiska_mobilitetKumulativaResultat.svg")

In [11]:

fig = go.Figure() 
#plots the admission data points
fig.add_trace(go.Scatter( 
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=datapoint_logistiska)), 
                    y=logistiskadata_tak['Inskrivningsdata'],
                    mode='markers',
                    name='Inskrivningsdata'))
#plots model admissions
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=datapoint_logistiska)), 
                    y=logistiskadata_tak['Optimala värden'],
                    mode='lines',
                    name='predikterade inskrivningar'))
#plots lower confidence interval
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=datapoint_logistiska)), 
                    y=logistiskadata_tak['Undre konfidens'],
                    mode='lines',
                    name='Undre konfidensintervall',
                    line = dict( width=2, dash='dash')))
#plots Upper confidence interval
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=datapoint_logistiska)), 
                    y=logistiskadata_tak['Övre konfidens'],
                    mode='lines',
                    name='Övre konfidensintervall',
                    line = dict(width=2, dash='dash')))
#plots lower prediction interval
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=datapoint_logistiska)), 
                    y=logistiskadata_tak['Undre prediktion'],
                    mode='lines',
                    name='Undre prediktionsintervall',
                    line = dict(width=2, dash='dot')))
#plots upper prediction interval
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=datapoint_logistiska)), 
                    y=logistiskadata_tak['Övre prediktion'],
                    mode='lines',
                    name='Övre prediktionsintervall',
                    line = dict(width=2, dash='dot')))
#plots RMSE annotation box
fig.add_annotation(x=pd.to_datetime("2020-03-01"),y=0, 
                   text=f"RMSE: {math.floor(mean_squared_error(logistiskadata_tak['Inskrivningsdata'],logistiskadata_tak['Optimala värden'], squared = False))}",
                   showarrow=False,
                   bordercolor="black",
                   xshift=48,
                   yshift=270
                  )
fig.update_layout(
                    title='Logistiska modellen med tak',
                    xaxis_title='Datum',
                    yaxis_title='Inskrivningar',
                    autosize=False,
                    width=1000,
                    xaxis_range=["2020-03-01",pd.to_datetime('2020-03-01')+timedelta(days=datapoint_logistiska)],
                    
                    font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="RebeccaPurple"
                    )
)
fig.show()
#fig.write_image("./logistiska_takKumulativaResultat.svg")

In [12]:

fig = go.Figure() 
#plots the admission data points
fig.add_trace(go.Scatter( 
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=datapoint_logistiska)), 
                    y=logistiskadata_mobilitet_tak['Inskrivningsdata'],
                    mode='markers',
                    name='Inskrivningsdata'))
#plots model admissions
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=datapoint_logistiska)), 
                    y=logistiskadata_mobilitet_tak['Optimala värden'],
                    mode='lines',
                    name='predikterade inskrivningar'))
#plots lower confidence interval
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=datapoint_logistiska)), 
                    y=logistiskadata_mobilitet_tak['Undre konfidens'],
                    mode='lines',
                    name='Undre konfidensintervall',
                    line = dict( width=2, dash='dash')))
#plots Upper confidence interval
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=datapoint_logistiska)), 
                    y=logistiskadata_mobilitet_tak['Övre konfidens'],
                    mode='lines',
                    name='Övre konfidensintervall',
                    line = dict(width=2, dash='dash')))
#plots lower prediction interval
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=datapoint_logistiska)), 
                    y=logistiskadata_mobilitet_tak['Undre prediktion'],
                    mode='lines',
                    name='Undre prediktionsintervall',
                    line = dict(width=2, dash='dot')))
#plots upper prediction interval
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="D", periods=datapoint_logistiska)), 
                    y=logistiskadata_mobilitet_tak['Övre prediktion'],
                    mode='lines',
                    name='Övre prediktionsintervall',
                    line = dict(width=2, dash='dot')))
#plots RMSE annotation box
fig.add_annotation(x=pd.to_datetime("2020-03-01"),y=0, 
                   text=f"RMSE: {math.floor(mean_squared_error(logistiskadata_mobilitet_tak['Inskrivningsdata'],logistiskadata_mobilitet_tak['Optimala värden'], squared = False))}",
                   showarrow=False,
                   bordercolor="black",
                   xshift=48,
                   yshift=270
                  )
fig.update_layout(
                    title='Logistiska modellen med mobilitet och tak',
                    xaxis_title='Datum',
                    yaxis_title='Inskrivningar',
                    autosize=False,
                    width=1000,
                    xaxis_range=["2020-03-01",pd.to_datetime('2020-03-01')+timedelta(days=datapoint_logistiska)],
                    
                    font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="RebeccaPurple"
                    )
)
fig.show()
#fig.write_image("./logistiska_mobilitet_takKumulativaResultat.svg")

In [13]:
dailyAdmissions = admission_data['Inskrivningar'][21:]

admission_figure1 = go.Figure()

admission_figure1.add_trace(go.Scatter(x=pd.date_range(start='2020-03-01',end='2020-07-31',freq='D'),
                                      y=admission_data['Inskrivningar'][21:]))

admission_figure1.update_layout(
                    title='Sahlgrenska inskrivningsdata',
                    xaxis_title='Datum',
                    yaxis_title='Inskrivningar',
                    autosize=False,
                    width=1000,
                    
                    font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="RebeccaPurple"
                    ))

admission_figure1.show()

dailyAdmissions = admission_data['Inskrivningar'][21:]

admission_figure2 = go.Figure()

admission_figure2.add_trace(go.Scatter(x=pd.date_range(start='2020-03-01',end='2020-07-31',freq='D'),
                                      y=np.cumsum(admission_data['Inskrivningar'][21:])))

admission_figure2.update_layout(
                    title='Sahlgrenska kumulative inskrivningsdata',
                    xaxis_title='Datum',
                    yaxis_title='Inskrivningar',
                    autosize=False,
                    width=1000,
                    
                    font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="RebeccaPurple"
                    ))

admission_figure2.show()